In [1]:
# import libraries
import pandas as pd
import time

In [2]:
# read data
data = pd.read_excel("Credit Card Payment Fraud Clean.xlsx")

In [3]:
# transform the data
data[["recordnum"]] = data[["recordnum"]].astype('category')
data[["cardnum"]] = data[["cardnum"]].astype('category')
data[["date"]] = data[["date"]].astype('datetime64[ns]')
data[["merchnum"]] = data[["merchnum"]].astype('category')
data[["merch.description"]] = data[["merch.description"]].astype('str')
data[["merch.state"]] = data[["merch.state"]].astype('category')
data[["merch.zip"]] = data[["merch.zip"]].astype('category')
data[["transtype"]] = data[["transtype"]].astype('category')
data[["amount"]] = data[["amount"]].astype('float')
data[["fraud"]] = data[["fraud"]].astype('category')

data=(data.sort_values(by="date")).reset_index(drop=True)

data.head()

,amount,cardnum,date,fraud,merch.description,merch.state,merch.zip,merchnum,recordnum,transtype
0,3.62,5142190439,2010-01-01,0,FEDEX SHP 12/23/09 AB#,TN,38118,5509006296254,1,P
1,3.80,5142148452,2010-01-01,0,FEDEX SHP 12/28/09 AB#,TN,38118,5509006296254,28,P
2,137.45,5142151402,2010-01-01,0,CDW*GOVERNMENT INC,IL,60061,5725000466504,29,P
3,3.62,5142135500,2010-01-01,0,FEDEX SHP 12/22/09 AB#,TN,38118,5509006296254,30,P
4,3.62,5142283088,2010-01-01,0,FEDEX SHP 12/22/09 AB#,TN,38118,5509006296254,31,P


In [4]:
data_test = data.loc[data["date"]<'2010-02-01',] # create a data to test on code

In [5]:
# Build Variables

In [6]:
## Type I Variable: unusual transaction frequency at both card and merchant level
def Build_A_Var(df, time_window, key):
    
    ###########################
    # df: the name of the cleaned data frame 
    # time_window: 3 or 7 or 14 or 28
    # key: "card" or "merchant"
    ###########################
    
    df["record"]=1
    df[key+"_"+"frequency_"+str(time_window)] = 1
    
    for i in range(df.shape[0]):
        current_date = df.loc[df.index==i,"date"].values[0]
        
        if key == "card":
            current_key = df.loc[df.index==i,"cardnum"].values[0]
            subset = df.loc[(df["cardnum"]==current_key) & (df["date"]<=current_date) \
                             & (df["date"]>=current_date-pd.Timedelta(days=time_window)) \
                             ,["cardnum","record"]]
            run_frequency = subset["record"].sum()
            df.loc[df.index==i,key+"_"+"frequency_"+str(time_window)] = run_frequency
        
        elif key == "merchant":
            current_key = df.loc[df.index==i,"merchnum"].values[0]
            subset = df.loc[(df["merchnum"]==current_key) & (df["date"]<=current_date) \
                             & (df["date"]>=current_date-pd.Timedelta(days=time_window)) \
                             ,["merchnum","record"]]
            run_frequency = subset["record"].sum()
            df.loc[df.index==i,key+"_"+"frequency_"+str(time_window)] = run_frequency
    del df["record"]
    
    return df

############################################
start = time.time()                        #
data_test = Build_A_Var(data_test,3,"card")#
end = time.time()                          #
print((end - start)/60)                    #         
############################################

/var/folders/9p/2vbggxfn5yv1xwsd0ycwf9240000gn/T/ipykernel_56221/3320619532.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["record"]=1
/var/folders/9p/2vbggxfn5yv1xwsd0ycwf9240000gn/T/ipykernel_56221/3320619532.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key+"_"+"frequency_"+str(time_window)] = 1


0.2378731369972229


In [7]:
## Type II Variable: unusual transaction amounts at both card and merchant level
def Build_B_Var(df, time_window, key):
    
    ###########################
    # df: the name of the cleaned data frame 
    # time_window: 3 or 7 or 14 or 28
    # key: "card" or "merchant"
    ###########################
    
    df[key+"_"+"amount_to_avg_"+str(time_window)] = 1
    df[key+"_"+"amount_to_max_"+str(time_window)] = 0
    df[key+"_"+"amount_to_median_"+str(time_window)] = 1
    df[key+"_"+"amount_to_total_"+str(time_window)] = 0
    
    
    for i in range(df.shape[0]):
        # print(i)
        current_date = df.loc[df.index==i,"date"].values[0]
        current_amount = df.loc[df.index==i,"amount"].values[0]
        
        if key == "card":
            current_key = df.loc[df.index==i,"cardnum"].values[0]
            subset = df.loc[(df["cardnum"]==current_key) & (df["date"]<current_date) \
                             & (df["date"]>=current_date-pd.Timedelta(days=time_window))  \
                             ,["cardnum","amount"]]
            
        elif key == 'merchant':
            current_key = df.loc[df.index==i,"merchnum"].values[0]
            subset = df.loc[(df["merchnum"]==current_key) & (df["date"]<current_date) \
                             & (df["date"]>=current_date-pd.Timedelta(days=time_window)) \
                             ,["merchnum","amount"]]
        # print(subset.shape[0])
        
        if subset.shape[0] != 0:
            run_avg = subset["amount"].mean()
            run_max = subset["amount"].max()
            run_median = subset["amount"].median()
            run_total = subset["amount"].sum()
            
            df.loc[df.index==i,key+"_"+"amount_to_avg_"+str(time_window)] = current_amount/run_avg
            df.loc[df.index==i,key+"_"+"amount_to_max_"+str(time_window)] = current_amount/run_max
            df.loc[df.index==i,key+"_"+"amount_to_median_"+str(time_window)] = current_amount/run_median
            df.loc[df.index==i,key+"_"+"amount_to_total_"+str(time_window)] = current_amount/run_total
    
    return df

############################################
start = time.time()                        #
data_test = Build_B_Var(data_test,3,"card")#
end = time.time()                          #
print((end - start)/60)                    #          
############################################ 

/var/folders/9p/2vbggxfn5yv1xwsd0ycwf9240000gn/T/ipykernel_56221/3523872715.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key+"_"+"amount_to_avg_"+str(time_window)] = 1
/var/folders/9p/2vbggxfn5yv1xwsd0ycwf9240000gn/T/ipykernel_56221/3523872715.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key+"_"+"amount_to_max_"+str(time_window)] = 0
/var/folders/9p/2vbggxfn5yv1xwsd0ycwf9240000gn/T/ipykernel_56221/3523872715.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

0.2935915152231852


In [8]:
## Type III Variable: unusual transaction location at both card and merchant level
def Build_C_Var(df, time_window, key):
    
    ###########################
    # df: the name of the cleaned data frame 
    # time_window: 3 or 7 or 14 or 28
    # key: "card" or "merchant"
    ###########################
    
    df[key+"_"+"distinct_state_"+str(time_window)] = 1
    df[key+"_"+"distinct_zip_"+str(time_window)] = 1
    
    for i in range(df.shape[0]):
        current_date = df.loc[df.index==i,"date"].values[0]
        
        if key == "card":
            current_key = df.loc[df.index==i,"cardnum"].values[0]
            subset = df.loc[(df["cardnum"]==current_key) & (df["date"]<=current_date) \
                             & (df["date"]>=current_date-pd.Timedelta(days=time_window)) \
                             ,["cardnum","merch.state","merch.zip"]]
            
            distinct_state = subset["merch.state"].nunique()
            distinct_zip = subset["merch.zip"].nunique()
    
            df.loc[df.index==i,key+"_"+"distinct_state_"+str(time_window)] = distinct_state
            df.loc[df.index==i,key+"_"+"distinct_zip_"+str(time_window)] = distinct_zip
        
        elif key == "merchant":
            current_key = df.loc[df.index==i,"merchnum"].values[0]
            subset = df.loc[(df["merchnum"]==current_key) & (df["date"]<=current_date) \
                             & (df["date"]>=current_date-pd.Timedelta(days=time_window)) \
                             ,["merchnum","merch.state","merch.zip"]]
            
            distinct_state = subset["merch.state"].nunique()
            distinct_zip = subset["merch.zip"].nunique()
    
            df.loc[df.index==i,key+"_"+"distinct_state_"+str(time_window)] = distinct_state
            df.loc[df.index==i,key+"_"+"distinct_zip_"+str(time_window)] = distinct_zip
    
    return df


################################################
start = time.time()                            #
data_test = Build_C_Var(data_test,3,"merchant")#
end = time.time()                              #
print((end - start)/60)                        #              
################################################ 

/var/folders/9p/2vbggxfn5yv1xwsd0ycwf9240000gn/T/ipykernel_56221/807171319.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key+"_"+"distinct_state_"+str(time_window)] = 1
/var/folders/9p/2vbggxfn5yv1xwsd0ycwf9240000gn/T/ipykernel_56221/807171319.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key+"_"+"distinct_zip_"+str(time_window)] = 1


0.27918575207392377


In [9]:
## Type IV Variable: unusual transaction interactions between card and merchant
def Build_D_Var(df, time_window, key):
    
    ###########################
    # df: the name of the cleaned data frame 
    # time_window: 3 or 7 or 14 or 28
    # key: "card" or "merchant"
    ###########################
    
    for i in range(df.shape[0]):
        current_date = df.loc[df.index==i,"date"].values[0]
        
        if key == "card":
            current_key = df.loc[df.index==i,"cardnum"].values[0]
            
            subset = df.loc[(df["cardnum"]==current_key) & (df["date"]<=current_date) \
                             & (df["date"]>=current_date-pd.Timedelta(days=time_window)) \
                             ,["cardnum","merchnum"]]
            
            distinct_merchnum = subset["merchnum"].nunique()
    
            df.loc[df.index==i,key+"_"+"distinct_merchnum_"+str(time_window)] = distinct_merchnum
         
        elif key == "merchant":
            current_key = df.loc[df.index==i,"merchnum"].values[0]
            
            subset = df.loc[(df["merchnum"]==current_key) & (df["date"]<=current_date) \
                             & (df["date"]>=current_date-pd.Timedelta(days=time_window)) \
                             ,["merchnum","cardnum"]]
            
            distinct_cardnum = subset["cardnum"].nunique()
    
            df.loc[df.index==i,key+"_"+"distinct_cardnum_"+str(time_window)] = distinct_cardnum
         
    return df

################################################
start = time.time()                            #
data_test = Build_D_Var(data_test,28,"card")   #
end = time.time()                              #
print((end - start)/60)                        #
################################################ 

/var/folders/9p/2vbggxfn5yv1xwsd0ycwf9240000gn/T/ipykernel_56221/46966338.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df.index==i,key+"_"+"distinct_merchnum_"+str(time_window)] = distinct_merchnum


0.24886916875839232


In [ ]:
start = time.time() 
for key in ["card","merchant"]:
    for time_window in [3,7,14,28]:
        print(key)
        print(time_window)
        data=Build_A_Var(data,time_window,key)
        data=Build_B_Var(data,time_window,key)
        data=Build_C_Var(data,time_window,key)
        data=Build_D_Var(data,time_window,key)
end = time.time()    
print((end - start)/60) 

In [ ]:
data.to_excel("Credit Card Payment Fraud Features.xlsx",index=False)